In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 2s (189 kB/s)
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

hs_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True, inferSchema=True)
hs_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
hs_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [6]:
# 2. Create a temporary view of the DataFrame.
hs_df.createOrReplaceTempView("home_sales")

In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT
    YEAR(date) as sales_year,
    ROUND(AVG(price),2) as avg_price
FROM home_sales
WHERE
    bedrooms = 4
GROUP BY sales_year
ORDER BY sales_year
"""
spark.sql(query).show()

+----------+---------+
|sales_year|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT
    date_built,
    ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE
    bedrooms = 4
    AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

spark.sql(query).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|287529.97|
|      2011|294656.14|
|      2012|289533.13|
|      2013|291870.89|
|      2014|289935.92|
|      2015| 299517.5|
|      2016|289090.11|
|      2017|289100.17|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT
    date_built,
    ROUND(AVG(price),2) as avg_price
FROM home_sales
WHERE
    bedrooms = 4
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

spark.sql(query).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|290237.14|
|      2011|293021.78|
|      2012|290024.97|
|      2013|293916.19|
|      2014|306202.48|
|      2015|305769.02|
|      2016|298295.66|
|      2017|289215.12|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT
    ROUND(AVG(view),2) as avg_view
FROM home_sales
WHERE
    price >= 350000
"""

spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.8914709091186523 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT
    ROUND(AVG(view),2) as avg_view
FROM home_sales
WHERE
    price >= 350000
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.28876709938049316 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
hs_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet("home_sales_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("p_home_sales")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
SELECT
    ROUND(avg(view),2) as avg_view
FROM p_home_sales
WHERE
    price >= 350000
"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.687523603439331 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False